In [1]:
#include <algorithm>
#include <bitset>
#include <iostream>
#include <sstream>
#include <utility>
#include <vector>

#include <xcpp/xdisplay.hpp>

#include <CArrayDefs/CArrayDefs.h>

namespace ht
{
    struct html
    {   
        inline html(const std::string& content)
        {
            m_content = content;
        }
        std::string m_content;
    };

    xeus::xjson mime_bundle_repr(const html& a)
    {
        auto bundle = xeus::xjson::object();
        bundle["text/html"] = a.m_content;
        return bundle;
    }
}

// Dump contents of 
template <typename T, typename V>
void dump_array(V a, const char *separator=" ", std::ostream &out=std::cout) {
    std::transform(a.data, a.data + a.length,
                   std::ostream_iterator<T>(out, separator), [](decltype(*a.data) v) {
                     return T(v);
                   });
}

template <typename T, typename V>
std::string concat_array(V a, const char *separator=" ") {
    std::stringstream output;
    dump_array<T>(a, separator, output);
    return output.str();
}

template <typename T, typename V>
auto display_array(V a, const char *separator=" ") {
    return ht::html(R"(
    <div>
    <style scoped="">
        .dataframe tbody tr th:only-of-type {
            vertical-align: middle;
        }

        .dataframe tbody tr th {
            vertical-align: top;
        }

        .dataframe thead th {
            text-align: right;
        }
    </style>
    <table border="1" class="dataframe">
      <tbody>
        <tr>
          <td>
             )" + concat_array<T>(a, "</td></tr><tr><td>") +
             R"(
          </td>
        </tr>
      </tbody>
    </table>
    </div>)");
}


struct Switch {
  uint8_t board;
  uint8_t port;
  uint8_t bit;
};

In [2]:
struct ChannelNeighbours {
  uint8_t up;
  uint8_t down;
  uint8_t left;
  uint8_t right;
};

In [3]:
std::vector<uint8_t> neighbours_raw(480);
std::vector<float> capacitances;
ChannelNeighbours *neighbours_begin;
std::vector<ChannelNeighbours> neighbours;

In [4]:
constexpr float C_THRESHOLD = 3e-12;

In [5]:
template <typename Neighbours, typename Capacitances>
std::vector<std::vector<int> > get_drops(Neighbours &neighbours, Capacitances &capacitances,
                                         float c_threshold=C_THRESHOLD) {
    std::vector<uint8_t> drop_member(neighbours.size(), 0xFF);

    for (auto i = 0; i < capacitances.size(); i++) {
        if (capacitances[i] <= C_THRESHOLD) {
            continue;
        }
        const auto &neighbours_i = neighbours[i];
        for (auto& x_i : std::vector<uint8_t>({static_cast<uint8_t>(i), neighbours_i.up, neighbours_i.down,
                                               neighbours_i.left, neighbours_i.right})) {
            if (x_i == 0xFF) { continue; }
            if (drop_member[x_i] < 0xFF) {
                const auto original_drop_i = drop_member[x_i];
                for (auto j = 0; j < drop_member.size(); j++) {
                    if (original_drop_i == drop_member[j]) {
                        drop_member[j] = i;
                    }
                }
            } else if (capacitances[x_i] > C_THRESHOLD) {
                drop_member[x_i] = i;
            }
        }
    }
    
    std::vector<std::vector<int> > drops;
    auto sorted = drop_member;
    std::sort(sorted.begin(), sorted.end());
    {
        auto end = std::unique(sorted.begin(), sorted.end());

        for (auto it = sorted.begin(); it != end -1; it++) {
            const auto drop_id_i = *it;
            std::vector<int> drop_i;

            auto j = 0;
            for (auto it_j = drop_member.begin(); it_j != drop_member.end(); it_j++, j++) {
                const auto drop_id_j = *it_j;
                if (drop_id_i == drop_id_j) {
                    drop_i.push_back(j);
                }
            }
            drops.push_back(drop_i);
        }
    }
    return drops;
}

In [6]:
// {40L: {40L, 42L, 45L, 74L, 77L}, 101L: {16L, 18L, 101L, 103L}}
capacitances = { 1.24667822e-12, 8.27001505e-13, 9.95153744e-13, 1.43268343e-12, 9.47640412e-13, 1.29708930e-13, 8.84316868e-13, 1.18850156e-12, 3.10656892e-13, 1.13740525e-12, 2.06812352e-13, 1.30152273e-12, 4.35249193e-13, 1.29722738e-12, 1.27270368e-12, 2.26044427e-14, 1.65408879e-11, 1.08673570e-12, 1.74066569e-11, 1.75067362e-13, 5.40601666e-13, 1.31088040e-12, 1.05582142e-12, 1.18255329e-12, 1.20343511e-12, 3.95570807e-13, 1.10969972e-13, 2.37078571e-14, 9.88628158e-13, 1.38852165e-12, 7.42327471e-13, 4.62978712e-13, 3.77267290e-13, 2.16225589e-13, 5.62731968e-13, 1.66790779e-14, 1.12166848e-12, 7.94581616e-13, 5.56852782e-13, 3.65395377e-13, 1.34614438e-11, 8.09808518e-13, 6.94362154e-12, 7.21385195e-14, 1.04109251e-12, 8.59589346e-12, 7.02285608e-13, 1.26616767e-12, 1.01989623e-12, 1.09170861e-12, 2.21547777e-13, 7.52629789e-13, 2.73809344e-13, 4.76071996e-13, 7.89181031e-14, 1.26721101e-12, 1.17025382e-12, 1.47050330e-14, 1.43388805e-12, 1.02922159e-12, 7.14176687e-13, 4.30621440e-13, 1.41806620e-12, 8.50601206e-13, 1.43132635e-12, 2.53628787e-13, 1.29381511e-12, 1.25666199e-13, 1.45989178e-12, 1.35720573e-13, 4.91186628e-13, 4.68155098e-13, 7.28003723e-13, 1.17302527e-13, 7.70999435e-12, 8.90601875e-13, 1.32843158e-12, 9.74318195e-12, 1.37122189e-12, 5.20137168e-13, 3.33946516e-13, 2.93442368e-13, 7.27743179e-13, 3.95209294e-13, 5.48976890e-14, 1.42610112e-12, 8.94287873e-13, 6.99918233e-13, 4.66592152e-13, 1.04319268e-13, 5.22581350e-13, 1.72636850e-13, 1.08398245e-12, 1.28676575e-12, 1.27055926e-12, 7.70763542e-13, 4.86687693e-14, 7.14150621e-13, 1.38567143e-12, 5.74144944e-13, 1.48952261e-12, 1.71236873e-11, 2.58594096e-15, 1.61987844e-11, 5.53589159e-13, 4.77702494e-13, 8.16773559e-13, 1.01204912e-12, 3.26579369e-13, 2.86208588e-13, 8.54996656e-13, 6.48789016e-13, 1.45638033e-12, 1.84571994e-13, 1.29371426e-12, 7.18470637e-13, 1.37153317e-12, 1.14362013e-12, 1.45752730e-12, 1.07865564e-12};
neighbours_raw = {255,   1,   2,   5,   0, 118,   2,   6, 255, 255,  30,   1, 255, 255, 255, 255,  10,   5, 255, 255,   4,   6,   0,  14,   5, 113, 1,   8, 255, 255, 255, 255,  14, 111,   6,  13, 255, 255,  10, 255,   9,   4, 255, 255, 255, 255, 255, 255, 255,  13,  14,  17, 12, 106,   8,  18, 255,   8,   5,  12, 255, 255, 255, 255,  22, 103,  18,  25, 255,  18,  12,  22,  17, 101,  13,  16, 255, 255, 255, 255,  26,  21, 255, 255,  20,  22, 255, 255,  21,  16,  17, 24, 255, 255, 255, 255, 255,  25,  22,  28,  24,  94,  16,  29, 255, 255,  20, 255, 255, 255, 255, 255, 255,  29,  24,  32,  28, 90,  25,  33, 255, 255,  58,   2, 255, 255, 255, 255, 255,  33, 28,  38,  32,  86,  29,  40, 255, 255,  36, 255, 255, 255, 255, 255,  34,  37, 255, 255,  36,  38, 255, 255,  37,  40,  32,  41, 255, 255, 255, 255,  38,  79,  33,  42, 255,  42,  38,  44,  41, 77,  40,  45, 255, 255, 255, 255, 255,  45,  41,  46,  44,  74, 42,  48, 255,  48,  44,  53, 255, 255, 255, 255,  46,  71,  45, 54, 255, 255,  50, 255,  49,  52, 255, 255, 255, 255, 255, 255, 50,  53, 255, 255,  52,  54,  46,  56,  53,  65,  48,  57, 255, 255, 255, 255, 255,  57,  53,  58,  56,  62,  54,  58, 255, 255, 56,  30, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,  62, 89,  57,  63,  65,  61,  62, 255,  66,  61, 255, 255, 255, 255, 54,  66,  71,  62,  65,  67,  73,  63,  66,  69, 255, 255, 255, 255, 255, 255,  67, 255,  70, 255,  69, 255, 255, 255,  48,  73, 74,  65, 255, 255, 255, 255,  71, 255,  75,  66,  45,  75,  77, 71,  74, 255,  78,  73, 255, 255, 255, 255,  42,  78,  79,  74, 77, 255,  81,  75,  40,  81,  86,  77, 255, 255, 255, 255,  79, 82,  87,  78,  81,  83, 255, 255,  82, 255,  85, 255, 255, 255, 255, 255,  83, 255, 255, 255,  33,  87,  90,  79,  86, 255,  91, 81, 255, 255, 255, 255, 255, 255,  61, 117,  29,  91,  94,  86, 90, 255,  95,  87, 255, 255, 255, 255,  99, 255, 255, 255,  25, 95, 103,  90,  94, 255,  97,  91, 255, 255, 255, 255, 103,  98, 102,  95,  97,  99, 255, 255,  98, 255,  93, 255, 255, 255, 255, 255,  18, 102, 106, 103, 101, 255, 107,  97,  16,  97, 101,  94, 255, 255, 255, 255, 111, 255, 114, 107,  13, 107, 111, 101, 106, 255, 105, 102, 255, 255, 255, 255, 115, 255, 110, 255, 109, 255, 255, 255,   8, 105, 113, 106, 255, 255, 255, 255,   6, 114, 118, 111, 113, 115, 119, 105, 114, 109, 255, 255, 255, 255, 255, 255, 255, 255,  89, 119,   1, 119, 117, 113, 118, 255, 117, 114};
neighbours_begin = reinterpret_cast<ChannelNeighbours *>(&neighbours_raw[0]);
neighbours = std::vector<ChannelNeighbours>(neighbours_begin, neighbours_begin + neighbours_raw.size()
                                            / sizeof(ChannelNeighbours));

{
    auto drops = get_drops(neighbours, capacitances, C_THRESHOLD);

    xcpp::display(ht::html("<h2>Drops (sets of connected channels with capacitance > 3 pF)</h2>"));
    for (auto it_drop = drops.begin(); it_drop != drops.end(); it_drop++) {
        auto &drop_i = *it_drop;
        xcpp::display(drop_i);
    }
}

Drops (sets of connected channels with capacitance > 3 pF)

{ 40, 42, 45, 74, 77 }

{ 16, 18, 101, 103 }

In [7]:
// Neighbours of electrodes
{
    auto drops = get_drops(neighbours, capacitances, C_THRESHOLD);
    std::vector<std::vector<uint8_t>> fringe;

    fringe.clear();

    for (auto it_drop = drops.begin(); it_drop != drops.end(); it_drop++) {
        std::vector<uint8_t> drop_neighbours(neighbours.size());
        std::fill(drop_neighbours.begin(), drop_neighbours.end(), 0xFF);
        for(auto &channel : *it_drop) {
            const auto &neighbours_i = neighbours[channel];
            for (auto& x_i : std::array<uint8_t, 4>({neighbours_i.up, neighbours_i.down,
                                                     neighbours_i.left, neighbours_i.right})) {
                drop_neighbours[x_i] = (*it_drop)[0];
            }
        }
        fringe.push_back(drop_neighbours);
    }

    auto sorted = fringe[0];
    for (auto i = 0; i < sorted.size(); i++) {
        if (sorted[i] < 0xFF) { sorted[i] = i; }
    }
    std::sort(sorted.begin(), sorted.end());
    xcpp::display(ht::html("<h2>Channels (including neighbours)</h2>"));
    xcpp::display(display_array<int>(UInt8Array_init(std::lower_bound(sorted.begin(),
                                                                      std::unique(sorted.begin(),
                                                                                  sorted.end()),
                                                                      0xFF) - sorted.begin(), &sorted[0])));
}

Channels (including neighbours)

33
38
40
41
42
44
45
48
71
74
75
